In [ ]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output

# Load the CSV data into a DataFrame
df = pd.read_csv("./aggregated_vaccine_data.csv")

In [ ]:


# Available years for dropdown options
years_available = [str(x) for x in df["Year"].unique()]

# Create a Dash app
app = Dash(__name__)

# Store previously selected years (initially empty list)
selected_years_prev = []

# App layout
app.layout = html.Div(
    [
        html.H1("Vaccination Comparison by Country"),
        html.Div(
            [
                html.Label("Select Year(s):"),
                dcc.Dropdown(
                    id="year-dropdown",
                    options=[{"label": y, "value": y} for y in years_available],
                    value=years_available,  # Pre-select all years initially
                    multi=True,
                ),
                html.Div(id="previous-selections", style={"font-weight": "bold"}),  # Element for previous selections
            ]
        ),
        dcc.Graph(id="vaccination-comparison-chart"),
    ]
)


@app.callback(
    Output("vaccination-comparison-chart", "figure"),
    Input("year-dropdown", "value"),
)
def update_graph(selected_years):
    global selected_years_prev  # Access global variable

    # Update previously selected years
    selected_years_prev = selected_years.copy()  # Avoid modifying original list

    # Filter data by selected years
    filtered_df = df[df["Year"].isin(selected_years)]
    print(filtered_df)

    # Create a new DataFrame to group data by country and year
    vaccine_counts_by_year = (
        filtered_df.groupby(["Country", "Year"])["Vaccinate"]
        .sum()
        .reset_index()
    )

    # Create a line chart using Matplotlib
    fig, ax = plt.subplots(figsize=(10, 6))
    for country in vaccine_counts_by_year["Country"].unique():
        country_data = vaccine_counts_by_year[
            vaccine_counts_by_year["Country"] == country
        ]
        plt.plot(country_data["Year"], country_data["Vaccinate"], label=country)

    plt.xlabel("Year")
    plt.ylabel("Number of Vaccinations")
    plt.title("Vaccination Comparison by Country")
    plt.legend()
    plt.xticks(rotation=45)  # Rotate x-axis labels for readability

    # Convert Matplotlib figure to Dash figure
    fig.update_layout(margin={"top": 20, "right": 20, "bottom": 30, "left": 50})
    return fig


@app.callback(
    Output("previous-selections", "children"),
    Input("year-dropdown", "value"),
)
def update_previous_selections(selected_years):
    if selected_years_prev:  # Check if previous selections exist
        return f"Previously Selected Years: {', '.join(selected_years_prev)}"
    else:
        return ""  # No previous selections to display


if __name__ == "__main__":
    app.run_server(debug=True,port=8099)


In [1]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import matplotlib.pyplot as plt
import seaborn as sns

# Sample data (replace with your actual data source)
data = {
    "Country": ["Germany", "France", "India", "USA", "Italy", "Algeria", "Argentina"],
    "Year": [2020, 2020, 2020, 2020, 2020, 2020, 2020],
    "Affected Cases": [10000, 15000, 20000, 25000, 30000, 5000, 7000]
}

df = pd.DataFrame(data)

# Available years for dropdown options
years_available = [str(x) for x in df["Year"].unique()]

# Create a Dash app
app = Dash(__name__)

# Store previously selected countries (initially empty list)
selected_countries_prev = []

# App layout
app.layout = html.Div(
    [
        html.H1("COVID-19 Affected Cases by Country (2020-2024)"),
        html.Div(
            [
                html.Label("Select Countries (Multi-Select):"),
                dcc.Dropdown(
                    id="country-dropdown",
                    options=[
                        {"label": country, "value": country}
                        for country in df["Country"].unique()
                    ],
                    value=df["Country"].unique(),  # Pre-select all countries initially
                    multi=True,
                ),
                html.Div(id="previous-selections", style={"font-weight": "bold"}),  # Element for previous selections
            ]
        ),
        dcc.Graph(id="covid-affected-cases-chart"),
    ]
)


@app.callback(
    Output("covid-affected-cases-chart", "figure"),
    Input("country-dropdown", "value"),
)
def update_graph(selected_countries):
    global selected_countries_prev  # Access global variable

    # Update previously selected countries
    selected_countries_prev = selected_countries.copy()

    # Filter data by selected countries
    filtered_df = df[df["Country"].isin(selected_countries)]

    # Create a line chart using Seaborn (integrated with Matplotlib)
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.lineplot(
        x="Year",
        y="Affected Cases",
        hue="Country",
        data=filtered_df,
        ax=ax,
        palette="Set3",  # Adjust color palette as needed
        style="o-",  # Marker style for data points
        linewidth=2,
    )

    ax.set_title("COVID-19 Affected Cases by Country (2020-2024)")
    ax.set_xlabel("Year")
    ax.set_ylabel("Affected Cases")
    ax.legend(title="Countries")

    return fig


@app.callback(
    Output("previous-selections", "children"),
    Input("country-dropdown", "value"),
)
def update_previous_selections(selected_countries):
    if selected_countries_prev:  # Check if previous selections exist
        return f"Previously Selected Countries: {', '.join(selected_countries_prev)}"
    else:
        return ""  # No previous selections to display


if __name__ == "__main__":
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.